In [ ]:
!pip install deeppavlov

In [ ]:
from deeppavlov import Element, Model
from deeppavlov.core.commands.utils import expand_path
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
from deeppavlov.download import download_resource
from deeppavlov.models.classifiers.proba2labels import Proba2Labels
from deeppavlov.utils.pip_wrapper.pip_wrapper import install_from_config

In [ ]:
classifiers_path = expand_path('~/.deeppavlov/models/classifiers')
model_path = classifiers_path / 'topic_distilbert_base_v0'
transformer_name = 'distilbert-base-uncased'
vocab_path = model_path / 'classes.dict'

In [ ]:
install_from_config('topics_distilbert_base_uncased')

download_resource(
    'http://files.deeppavlov.ai/deeppavlov_data/classifiers/topic_distilbert_base_v0.tar.gz',
    {classifiers_path}
)

In [ ]:
from deeppavlov.models.preprocessors.torch_transformers_preprocessor import TorchTransformersPreprocessor
from deeppavlov.models.torch_bert.torch_transformers_classifier import TorchTransformersClassifierModel

In [ ]:
preprocessor = TorchTransformersPreprocessor(vocab_file=transformer_name, max_seq_length=128)

classes_vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)

classifier = TorchTransformersClassifierModel(
    n_classes=classes_vocab.len,
    return_probas=True,
    pretrained_bert=transformer_name,
    save_path=model_path / 'model',
    optimizer_parameters={'lr': 1e-05}
)

proba2labels = Proba2Labels(max_proba=True)

model = Model(
    x=['x'],
    out=['y_pred_labels'],
    pipe=[
        Element(component=preprocessor, x=['x'], out=['bert_features']),
        Element(component=classifier, x=['bert_features'], out=['y_pred_probas']),
        Element(component=proba2labels, x=['y_pred_probas'], out=['y_pred_ids']),
        Element(component=classes_vocab, x=['y_pred_ids'], out=['y_pred_labels'])
    ]
)

In [ ]:
model(['What do you think about Arrival with Amy Adams'])